In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [7]:
from llama_index.core import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [9]:
from utils import build_sentence_window_index

from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [10]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [11]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'You create your AI portfolio by building a collection of projects that demonstrate a progression of skills in AI.'

## Feedback functions

In [12]:
import nest_asyncio

nest_asyncio.apply()

In [13]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [14]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [16]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [17]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [18]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [20]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [21]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [23]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [24]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [25]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [26]:
eval_questions.append("How can I be successful in AI?")

In [27]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [28]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [29]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_094da93a94260ebcaf110763c49b69b6,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_094da93a94260ebcaf1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-03-02T06:54:56.256515"", ""...",2024-03-02T06:55:05.184813,0.8,0.50,0.8,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 6: Building a Po...,[{'args': {'question': 'What are the keys to b...,8,540,0.000825
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_87ce11a6cbcb559356a39ccd6087c9e6,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_87ce11a6cbcb559356a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-03-02T06:55:05.760818"", ""...",2024-03-02T06:55:13.407118,1.0,0.55,0.5,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,7,707,0.001097
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_4c3f0efd514dd2ee34ca422b3ec0ee37,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",-,"{""record_id"": ""record_hash_4c3f0efd514dd2ee34c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-03-02T06:55:13.678095"", ""...",2024-03-02T06:55:19.596597,0.9,0.25,0.7,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,5,602,0.000963
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_ee94a0da6bc7c6cbf27d582116e8eca6,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_ee94a0da6bc7c6cbf27...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-03-02T06:55:19.868685"", ""...",2024-03-02T06:55:23.027816,1.0,0.50,0.5,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,3,548,0.000840
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_99f8fe3496abcacde7d792304299469b,"""How can altruism be beneficial in building a ...","""Helping others and aiming to lift them during...",-,"{""record_id"": ""record_hash_99f8fe3496abcacde7d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-03-02T06:55:23.359847"", ""...",2024-03-02T06:55:26.761858,0.9,1.00,NaN,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,NaN,3,509,0.000774


In [30]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Groundedness,Answer Relevance
0,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.""",0.80,0.500000,0.8
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and perspectives of their colleagues. Collaborating with others can lead to more innovative solutions, effective problem-solving, and a more comprehensive understanding of complex AI projects. Additionally, working in a team can help individuals learn from each other, share knowledge, and collectively drive the project forward towards successful outcomes.""",0.50,0.550000,1.0
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, learn about different job roles, and receive advice on how to navigate their career paths effectively. Networking also allows individuals to build relationships with others in the field, which can lead to potential collaborations, mentorship opportunities, and even job referrals. Overall, networking in AI can help individuals stay informed, expand their professional connections, and enhance their career prospects in the rapidly evolving field of artificial intelligence.""",0.70,0.250000,0.9
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health.""",0.50,0.500000,1.0
4,"""How can altruism be beneficial in building a career?""","""Helping others and aiming to lift them during one's career journey can lead to better outcomes for oneself.""",NaN,1.000000,0.9
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological pattern where individuals doubt their accomplishments and have a persistent fear of being exposed as a fraud. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community despite their success. This can lead to self-doubt and hinder their growth in the field.""",0.90,0.233333,1.0
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",0.95,0.500000,1.0
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",0.75,1.000000,1.0
8,"""What are some common challenges in AI?""","""Common challenges in AI include the evolving nature of technologies, the need to stay updated with rapidly changing technology, the difficulty in estimating project timelines and returns on investment, the iterative nature of AI projects leading to uncertainties in project management, the challenge of collaborating with stakeholders lacking AI expertise, and the inconsistency in opinions on AI skills and job roles.""",0.50,NaN,1.0
9,"""Is it normal to find parts of AI challenging?""","""It is normal to find parts of AI challenging.""",NaN,1.000000,1.0


In [34]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
App_1,0.7,0.684848,0.954545,3.909091,0.000911


In [35]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

RuntimeError: Dashboard failed to start in time. Please inspect dashboard logs for additional information.